In [1]:
import os
import glob
import numpy as np
import pandas as pd
import nibabel as nib
import scipy.io as sio
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.multivariate.manova import MANOVA
from sklearn.svm import SVC
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
ukb_smri_data_path = "/Users/xli77/Documents/MIVA/output/UKB_MMIVA_C30_preregSite_SMRI_MancovanOuts_wX_FINAL.mat"
ukb_smri_data = sio.loadmat(ukb_smri_data_path)['MODELUKB0s_ful']
ukb_smri_data_array = ukb_smri_data[0][0][0]
ukb_smri_data_key = np.squeeze(ukb_smri_data[0][0][3])
age_idx = np.where(ukb_smri_data_key==['age_when_attended_assessment_centre_f21003_2_0'])[0][0]
sex_idx = np.where(ukb_smri_data_key==['sex_f31_0_0'])[0][0]
age = ukb_smri_data_array[:, age_idx]
sex = ukb_smri_data_array[:, sex_idx]
age_sex = age * sex

In [3]:
datapath="/Users/xli77/Documents/MISA/results/SIVA/fixedSubspace/um2mm/"
subspace_struct_list=['234111','2222211','333111','441111']

Y = np.zeros((4,2,2,12,2907)) # S1-4, UA/MSIVA, M1-2, voxel, source

num_iter=21

for i,ss in enumerate(subspace_struct_list):

    data=sio.loadmat(os.path.join(datapath,f"subspace_struct_{ss}","um_neuroimaging_Y.mat"))
    Y1=np.squeeze(data['Y1'])

    data=sio.loadmat(os.path.join(datapath,f"subspace_struct_{ss}","ummm_neuroimaging_Y.mat"))
    Y2=np.squeeze(data['Y2'])

    Y[i,0,0]=Y1[0]
    Y[i,0,1]=Y1[1]
    Y[i,1,0]=Y2[0]
    Y[i,1,1]=Y2[1]

In [4]:
A = sio.loadmat("/Users/xli77/Documents/MISA/results/SIVA/fixedSubspace/mask/A.mat")["A"] # S1-4, UA/MSIVA, M1-2, voxel, source
A1 = A[1,1,0,:,:] # structure 2, MSIVA, M1, subspace 1
A2 = A[1,1,1,:,:] # structure 2, MSIVA, M2, subspace 1

A.shape, A1.shape, A2.shape

((4, 2, 2, 44318, 12), (44318, 12), (44318, 12))

In [10]:
# S2 subspace across two modalities
predictor = np.concatenate((np.expand_dims(age,axis=1),np.expand_dims(sex,axis=1),np.expand_dims(age_sex,axis=1)), axis=1)
response = np.concatenate((Y[1,1,0,:], Y[1,1,1,:]), axis=0).T
print(predictor.shape, response.shape)

(2907, 3) (2907, 24)


In [5]:
def zscore(data, axis):
    data -= data.mean(axis=axis, keepdims=True)
    data /= data.std(axis=axis, keepdims=True)
    return np.nan_to_num(data, copy=False)

def correlation(matrix1, matrix2):
    d1 = matrix1.shape[-1]
    d2 = matrix2.shape[-1]

    assert d1 == d2
    assert matrix1.ndim <= 2
    assert matrix2.ndim <= 2
    
    matrix1 = zscore(matrix1.astype(float), matrix1.ndim - 1) / np.sqrt(d1)
    matrix2 = zscore(matrix2.astype(float), matrix2.ndim - 1) / np.sqrt(d2)
    
    if matrix1.ndim >= matrix2.ndim:
        return np.dot(matrix1, matrix2.T)
    else:
        return np.dot(matrix2, matrix1.T)

In [11]:
predictor = np.concatenate((np.expand_dims(age,axis=1),np.expand_dims(sex,axis=1),np.expand_dims(age_sex,axis=1)), axis=1)
data = np.concatenate((predictor, response), axis=1)

data_name = ['Age', 'Sex', 'Age_Sex']
for i in range(2):
    for j in range(12):
        data_name.append(f'M{i+1}SCV{j+1}')

df = pd.DataFrame(data, columns=data_name)
df.head(5)

,Age,Sex,Age_Sex,M1SCV1,M1SCV2,M1SCV3,M1SCV4,M1SCV5,M1SCV6,M1SCV7,...,M2SCV3,M2SCV4,M2SCV5,M2SCV6,M2SCV7,M2SCV8,M2SCV9,M2SCV10,M2SCV11,M2SCV12
0,52.0,1.0,52.0,-0.744373,-0.014831,2.725441,-3.130108,-0.743085,0.711263,0.982630,...,0.905101,-4.688589,1.545894,0.862613,-0.382749,1.233576,0.360367,-0.948850,-0.740604,-1.480718
1,58.0,0.0,0.0,3.849157,0.788538,-0.189606,5.001522,1.692831,1.251061,0.305623,...,-0.986274,0.381924,-2.058605,3.226054,2.865676,-0.503773,-0.070533,1.720351,0.042034,0.773175
2,63.0,1.0,63.0,2.019567,0.641578,0.218308,-2.097338,-1.564189,-1.111780,3.238668,...,-1.819940,-3.231881,-1.349841,2.612795,-2.174093,-0.221382,-2.810719,-1.601670,-0.252891,0.646500
3,68.0,1.0,68.0,1.090513,0.363112,1.388326,1.629931,-0.339085,2.347692,-0.641168,...,-1.376372,-0.249094,-0.474632,-1.392171,1.123885,-1.742248,-2.902886,-0.192690,-0.610004,-2.534472
4,67.0,1.0,67.0,1.096157,4.176810,0.261035,-1.457208,-1.231453,-2.161218,1.134822,...,1.465951,0.378329,0.638157,0.011793,0.712127,1.017233,-1.744040,-4.447122,-1.297760,-1.592874


In [6]:
def age_regression(X_train, X_test, y_train, y_test):
    clf = Ridge(alpha=1) 
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    mae = np.mean(np.abs(y_pred-y_test))
    return mae

def sex_classification(X_train, X_test, y_train, y_test):
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = 1 - np.mean(np.abs(y_pred-y_test))
    return acc

In [8]:
i = 0
Y1 = np.concatenate((Y[1,1,0,i:i+2], Y[1,1,1,i:i+2]), axis=0).T
A1 = np.concatenate((A[1,1,0,:,i:i+2], A[1,1,1,:,i:i+2]), axis=1).T 
X1 = Y1 @ A1
Y1.shape, A1.shape

((2907, 4), (4, 44318))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X1, age, test_size=0.312, random_state=42)

In [14]:
# SVC
param_grid= {'kernel': ('linear', 'rbf'),
             'C': [1, 10]}
base_estimator = SVC(gamma='scale')
svc = GridSearchCV(base_estimator, param_grid, cv=5).fit(X_train, y_train)
svc.best_estimator_

/Users/xli77/miniconda3/envs/nhpseg/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


SVC(C=1, kernel='linear')

In [13]:
# Ridge
param_grid= {'alpha': [0.1, 0.3, 0.5, 0.7, 1]}
base_estimator = Ridge()
rr = GridSearchCV(base_estimator, param_grid, cv=5).fit(X_train, y_train)
rr.best_estimator_

Ridge(alpha=1)

In [14]:
# LASSO
param_grid= {'alpha': [0.1, 0.3, 0.5, 0.7, 1]}
base_estimator = Lasso()
lr = GridSearchCV(base_estimator, param_grid, cv=5).fit(X_train, y_train)
lr.best_estimator_

Lasso(alpha=0.1)

In [18]:
for i in np.arange(0,10,2):
    
    # TODO
    # cross-validation, 1k test, 10-fold cross-val on 2k
    # tune hyperparameters for ridge/LASSO regression 

    print(f"Subspace {i//2+1}")

    # unimodal
    Y1 = np.concatenate((Y[1,0,0,i:i+2], Y[1,0,1,i:i+2]), axis=0).T
    A1 = np.concatenate((A[1,0,0,:,i:i+2], A[1,0,1,:,i:i+2]), axis=1).T 
    X1 = Y1 @ A1
    # X1 = A1 @ Y1 

    # multimodal
    Y2 = np.concatenate((Y[1,1,0,i:i+2], Y[1,1,1,i:i+2]), axis=0).T
    A2 = np.concatenate((A[1,1,0,:,i:i+2], A[1,1,1,:,i:i+2]), axis=1).T 
    X2 = Y2 @ A2
    # X2 = A2 @ Y2 

    # X1 = np.concatenate((Y[1,0,0,i:i+2], Y[1,0,1,i:i+2]), axis=0).T
    # X2 = np.concatenate((Y[1,1,0,i:i+2], Y[1,1,1,i:i+2]), axis=0).T
    
    X_train, X_test, y_train, y_test = train_test_split(X1, age, test_size=0.312, random_state=42)
    mae = age_regression(X_train, X_test, y_train, y_test)
    
    X_train, X_test, y_train, y_test = train_test_split(X2, age, test_size=0.312, random_state=42)
    mae2 = age_regression(X_train, X_test, y_train, y_test)
    print(f"Age Regression MAE  UA: {round(mae, 4)}, MA: {round(mae2, 4)}")

    X_train, X_test, y_train, y_test = train_test_split(X1, sex, test_size=0.312, random_state=42)
    acc = sex_classification(X_train, X_test, y_train, y_test)

    X_train, X_test, y_train, y_test = train_test_split(X2, sex, test_size=0.312, random_state=42)
    acc2 = sex_classification(X_train, X_test, y_train, y_test)
    print(f"Sex Classification Accuracy  UA: {round(acc, 4)}, MA: {round(acc2, 4)}")


Subspace 1
Age Regression MAE  UA: 5.6883, MA: 5.6405
Sex Classification Accuracy  UA: 0.6289, MA: 0.5876
Subspace 2
Age Regression MAE  UA: 5.6774, MA: 6.1725
Sex Classification Accuracy  UA: 0.5653, MA: 0.61
Subspace 3
Age Regression MAE  UA: 5.8381, MA: 5.9648
Sex Classification Accuracy  UA: 0.7595, MA: 0.5842
Subspace 4
Age Regression MAE  UA: 6.1548, MA: 5.8931
Sex Classification Accuracy  UA: 0.5859, MA: 0.8024
Subspace 5
Age Regression MAE  UA: 5.7194, MA: 5.2504
Sex Classification Accuracy  UA: 0.6014, MA: 0.5172


In [10]:
# evaluate first subspace 
fit = MANOVA.from_formula('M1SCV1 + M1SCV2 + M2SCV1 + M2SCV2 ~ Age + Sex + Age_Sex', data=df)
print(fit.mv_test())

                  Multivariate linear model
                                                              
--------------------------------------------------------------
       Intercept        Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9517 4.0000 2900.0000 36.7786 0.0000
         Pillai's trace 0.0483 4.0000 2900.0000 36.7786 0.0000
 Hotelling-Lawley trace 0.0507 4.0000 2900.0000 36.7786 0.0000
    Roy's greatest root 0.0507 4.0000 2900.0000 36.7786 0.0000
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
          Age           Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9450 4.0000 2900.0000 42.2087 0.0000
         Pillai's trace 0.0550 4.0000 2900.0000 42.2087 0.0000
 Hotelling-

In [11]:
# evaluate second subspace 
fit = MANOVA.from_formula('M1SCV3 + M1SCV4 + M2SCV3 + M2SCV4 ~ Age + Sex + Age_Sex', data=df)
print(fit.mv_test())

                  Multivariate linear model
                                                              
--------------------------------------------------------------
       Intercept        Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9277 4.0000 2900.0000 56.4686 0.0000
         Pillai's trace 0.0723 4.0000 2900.0000 56.4686 0.0000
 Hotelling-Lawley trace 0.0779 4.0000 2900.0000 56.4686 0.0000
    Roy's greatest root 0.0779 4.0000 2900.0000 56.4686 0.0000
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
          Age           Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9356 4.0000 2900.0000 49.8773 0.0000
         Pillai's trace 0.0644 4.0000 2900.0000 49.8773 0.0000
 Hotelling-

In [12]:
# evaluate third subspace 
fit = MANOVA.from_formula('M1SCV5 + M1SCV6 + M2SCV5 + M2SCV6 ~ Age + Sex + Age_Sex', data=df)
print(fit.mv_test())

                  Multivariate linear model
                                                              
--------------------------------------------------------------
       Intercept        Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9634 4.0000 2900.0000 27.5207 0.0000
         Pillai's trace 0.0366 4.0000 2900.0000 27.5207 0.0000
 Hotelling-Lawley trace 0.0380 4.0000 2900.0000 27.5207 0.0000
    Roy's greatest root 0.0380 4.0000 2900.0000 27.5207 0.0000
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
          Age           Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9400 4.0000 2900.0000 46.2604 0.0000
         Pillai's trace 0.0600 4.0000 2900.0000 46.2604 0.0000
 Hotelling-

In [13]:
# evaluate forth subspace 
fit = MANOVA.from_formula('M1SCV7 + M1SCV8 + M2SCV7 + M2SCV8 ~ Age + Sex + Age_Sex', data=df)
print(fit.mv_test())

                  Multivariate linear model
                                                              
--------------------------------------------------------------
       Intercept        Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9834 4.0000 2900.0000 12.2512 0.0000
         Pillai's trace 0.0166 4.0000 2900.0000 12.2512 0.0000
 Hotelling-Lawley trace 0.0169 4.0000 2900.0000 12.2512 0.0000
    Roy's greatest root 0.0169 4.0000 2900.0000 12.2512 0.0000
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
          Age           Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9864 4.0000 2900.0000  9.9912 0.0000
         Pillai's trace 0.0136 4.0000 2900.0000  9.9912 0.0000
 Hotelling-

In [14]:
# evaluate fifth subspace 
fit = MANOVA.from_formula('M1SCV9 + M1SCV10 + M2SCV9 + M2SCV10 ~ Age + Sex + Age_Sex', data=df)
print(fit.mv_test())

                  Multivariate linear model
                                                              
--------------------------------------------------------------
       Intercept        Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9050 4.0000 2900.0000 76.0687 0.0000
         Pillai's trace 0.0950 4.0000 2900.0000 76.0687 0.0000
 Hotelling-Lawley trace 0.1049 4.0000 2900.0000 76.0687 0.0000
    Roy's greatest root 0.1049 4.0000 2900.0000 76.0687 0.0000
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
          Age           Value  Num DF   Den DF  F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.9089 4.0000 2900.0000 72.6787 0.0000
         Pillai's trace 0.0911 4.0000 2900.0000 72.6787 0.0000
 Hotelling-

In [15]:
fit = smf.glm(formula='M1SCV11 ~ Age + Sex + Age_Sex', data=df).fit()
print(fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                M1SCV11   No. Observations:                 2907
Model:                            GLM   Df Residuals:                     2903
Model Family:                Gaussian   Df Model:                            3
Link Function:               identity   Scale:                          2.5273
Method:                          IRLS   Log-Likelihood:                -5470.5
Date:                Tue, 25 Oct 2022   Deviance:                       7336.8
Time:                        17:38:34   Pearson chi2:                 7.34e+03
No. Iterations:                     3   Pseudo R-squ. (CS):            0.06912
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.7667      0.355    -10.620      0.0

In [16]:
fit = smf.glm(formula='M1SCV12 ~ Age + Sex + Age_Sex', data=df).fit()
print(fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                M1SCV12   No. Observations:                 2907
Model:                            GLM   Df Residuals:                     2903
Model Family:                Gaussian   Df Model:                            3
Link Function:               identity   Scale:                          2.7444
Method:                          IRLS   Log-Likelihood:                -5590.2
Date:                Tue, 25 Oct 2022   Deviance:                       7966.9
Time:                        17:38:34   Pearson chi2:                 7.97e+03
No. Iterations:                     3   Pseudo R-squ. (CS):            0.01431
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9202      0.370      5.196      0.0

In [17]:
fit = smf.glm(formula='M2SCV11 ~ Age + Sex + Age_Sex', data=df).fit()
print(fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                M2SCV11   No. Observations:                 2907
Model:                            GLM   Df Residuals:                     2903
Model Family:                Gaussian   Df Model:                            3
Link Function:               identity   Scale:                          2.7349
Method:                          IRLS   Log-Likelihood:                -5585.2
Date:                Tue, 25 Oct 2022   Deviance:                       7939.3
Time:                        17:38:34   Pearson chi2:                 7.94e+03
No. Iterations:                     3   Pseudo R-squ. (CS):           0.005135
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1133      0.369     -0.307      0.7

In [18]:
fit = smf.glm(formula='M2SCV12 ~ Age + Sex + Age_Sex', data=df).fit()
print(fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                M2SCV12   No. Observations:                 2907
Model:                            GLM   Df Residuals:                     2903
Model Family:                Gaussian   Df Model:                            3
Link Function:               identity   Scale:                          2.6013
Method:                          IRLS   Log-Likelihood:                -5512.4
Date:                Tue, 25 Oct 2022   Deviance:                       7551.7
Time:                        17:38:34   Pearson chi2:                 7.55e+03
No. Iterations:                     3   Pseudo R-squ. (CS):            0.01340
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.4159      0.360     -3.935      0.0